In [1]:
from MAUtil import *
from MAInit import *
import time

In [2]:
#############################################################################3
# name = 'Au_111_u2'
# name = 'Au_100_u2'
# name = 'Pd_111_u2'
# name = 'Pd_100_u2'
# name = 'Al_111_u2'
# name = 'Al_100_u2'
# name = 'Ni_111_u2'
# name = 'Ni_100_u1'
# name = 'Ru_0001_u2'
name = 'Zn_0001_u2'
# name = 'Rh_111_u2'
# name = 'Rh_100_u2'
name = 'Ag_100_u2_no03_CO_n3_d3'
name = 'Cu_100_u2_RPBE'

In [3]:
sur = make_adsorbed_surface('Cu_100_u2_RPBE', 'CO')
sur.make_surface(maxmole=2, mindist=0.0001)

AttributeError: 'str' object has no attribute 'symbols'

In [4]:
sur.write_trajectory()

Cu_100_u2_RPBE_no001_CO_n1_d8.traj
Cu_100_u2_RPBE_no002_CO_n2_d3.traj
Cu_100_u2_RPBE_no003_CO_n2_d5.traj
Cu_100_u2_RPBE_no004_CO_n2_d3.traj
Cu_100_u2_RPBE_no005_CO_n2_d1.traj
Cu_100_u2_RPBE_no006_CO_n2_d0.traj
Cu_100_u2_RPBE_no007_CO_n1_d8.traj
Cu_100_u2_RPBE_no008_CO_n2_d1.traj
Cu_100_u2_RPBE_no009_CO_n2_d0.traj
Cu_100_u2_RPBE_no010_CO_n2_d3.traj
Cu_100_u2_RPBE_no011_CO_n2_d5.traj
Cu_100_u2_RPBE_no012_CO_n2_d3.traj
Cu_100_u2_RPBE_no013_CO_n2_d3.traj
Cu_100_u2_RPBE_no014_CO_n1_d8.traj
Cu_100_u2_RPBE_no015_CO_n2_d3.traj
Cu_100_u2_RPBE_no016_CO_n2_d5.traj
